In [ ]:
import pandas as pd
from plotly import graph_objects as go
import nevergrad as ng
from summer2 import AgeStratification, Overwrite

from estival.wrappers.nevergrad import optimize_model
import estival.priors as esp
import estival.targets as est
from estival.model import BayesianCompartmentalModel
from estival.wrappers import pymc as epm
from summer2 import CompartmentalModel
from summer2.parameters import Parameter, Function, DerivedOutput
from summer2.functions.time import get_sigmoidal_interpolation_function

from tb_incubator.demographics import add_extra_crude_birth_flow
from tb_incubator.constants import set_project_base_path
from tb_incubator.input import get_birth_rate, get_pop_death_data, get_death_rates

pd.options.plotting.backend = "plotly"
project_paths = set_project_base_path("../tb_incubator/")

In [ ]:
# load birth data
birth_rates = get_birth_rate()

In [ ]:
# load age-stratified, population & deaths data
pop_death = get_pop_death_data()
target_pops = pop_death.groupby(level=[0]).sum()["population"]

In [ ]:
# load death rates
death_rates = get_death_rates()

In [ ]:
death_rates.plot()

In [ ]:
# Population per year
total_population_per_year = pop_death.groupby('year')['population'].sum()

In [ ]:
# Arbitrary base model construction
model_comps = ["susceptible", "early latent", "late latent", "infectious", "recovered"]
model_times = [1850.0, 2024.0]
model = CompartmentalModel(
    times=model_times,
    compartments=model_comps,
    infectious_compartments=["infectious"],
)
init_pops = {"susceptible": Parameter("starting population"), "infectious": 0.0}
model.set_initial_population(init_pops)

In [ ]:
# TB transitions, some meaningless TB-related flows
model.add_death_flow("TB death", Parameter("death rate"), "infectious")

In [ ]:
# Demographic transitions
model.add_universal_death_flows("population_death", 1.0)
model.add_replacement_birth_flow("replacement_birth", "susceptible")

In [ ]:
pop_start_year = total_population_per_year.index[0]
start_period = pop_start_year - model_times[0]

In [ ]:
total_population_change = total_population_per_year.diff().dropna()

In [ ]:
total_population_change[1950] = total_population_per_year[1950] / start_period

In [ ]:
total_population_change = total_population_change.sort_index()

In [ ]:
entry_rate = get_sigmoidal_interpolation_function(
    total_population_change.index, 
    total_population_change,
)

In [ ]:
# Arbitrary epidemiological parameter
params = {"death rate": 0.01}

In [ ]:
# Apply age stratification with age-specific death rate functions of time
agegroup_request = [[0, 4], [5, 14], [15, 34], [35, 49], [50, 100]]
age_strata = [i[0] for i in agegroup_request]
strat = AgeStratification("age", age_strata, model_comps)
death_adjs = {}
for age in age_strata:
    years = death_rates.index
    rates = death_rates[age]
    pop_death_func = get_sigmoidal_interpolation_function(years, rates)
    death_adjs[str(age)] = Overwrite(pop_death_func)
strat.set_flow_adjustments("population_death", death_adjs)
model.stratify_with(strat)

In [ ]:
model.add_importation_flow("births", entry_rate, dest="susceptible", split_imports=True, dest_strata={"age": "0"})

In [ ]:
# Track population
model.request_output_for_compartments("total_population", model_comps)
for s in age_strata:
    model.request_output_for_compartments(f"{s}_population", model_comps, strata={"age": str(s)})

In [ ]:
model.run({"starting population": 0.0, "death rate": 0.01})
age_pops = model.get_derived_outputs_df()[[f"{s}_population" for s in age_strata]]
age_pops.plot.area()

In [ ]:
# Inspect outputs
outputs = model.get_derived_outputs_df()
fig = go.Figure()
fig.add_trace(go.Scatter(x=outputs.index, y=outputs["total_population"], name="modelled"))
fig.add_trace(go.Scatter(x=target_pops.index, y=target_pops, name="target", mode="markers", marker=dict(color="black", size=2.0)))
fig.update_layout(yaxis={"range": [0.0, 4e8]})